# Product SVD in Python


In this NoteBook, the reader will find code to load GeoTiff files, single- or multi-band, from HDFS. It reads the GeoTiffs as a **ByteArray**s and then stores the GeoTiffs in memory using **MemFile** from the **RasterIO** Python package. Subsequently, a statistical analysis is performed on each pair of datasets. In particular, the Python module _productsvd_ is used to determine the SVD of the product of the two phenology datasets.

## Initialization
This section initializes the notebook.

### Dependencies
Here, all necessary libraries are imported.

In [1]:
#Add all dependencies to PYTHON_PATH
import sys
sys.path.append("/usr/lib/spark/python")
sys.path.append("/usr/lib/spark/python/lib/py4j-0.10.4-src.zip")
sys.path.append("/usr/lib/python3/dist-packages")
sys.path.append("/data/local/jupyterhub/modules/python")

#Define environment variables
import os
os.environ["HADOOP_CONF_DIR"] = "/etc/hadoop/conf"
os.environ["PYSPARK_PYTHON"] = "python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = "ipython"

import subprocess

#Load PySpark to connect to a Spark cluster
from pyspark import SparkConf, SparkContext
from hdfs import InsecureClient
from tempfile import TemporaryFile

#from osgeo import gdal
#To read GeoTiffs as a ByteArray
from io import BytesIO
from rasterio.io import MemoryFile

import numpy as np
import pandas
import datetime
import matplotlib.pyplot as plt
import rasterio
from rasterio import plot
from os import listdir
from os.path import isfile, join
from numpy import exp, log
from numpy.random import standard_normal
import scipy.linalg
from productsvd import qrproductsvd
from sklearn.utils.extmath import randomized_svd

### Configuration
This configuration determines whether functions print logs during the execution.

In [2]:
debugMode = True
maxModes = 3

### Connect to Spark
Here, the Spark context is loaded, which allows for a connection to HDFS.

In [3]:
appName = "plot_GeoTiff"
masterURL = "spark://emma0.emma.nlesc.nl:7077"

#A context needs to be created if it does not already exist
try:
    sc.stop()
except NameError:
    print("A new Spark Context will be created.")

sc = SparkContext(conf = SparkConf().setAppName(appName).setMaster(masterURL))
conf = sc.getConf()

A new Spark Context will be created.


## Functions
This section defines various functions used in the analysis.

### Support functions
These functions support other functions.

In [4]:
def dprint(msg):
    if (debugMode):
        print(str(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")) + " | " + msg)

In [5]:
def progressBar(message, value, endvalue, bar_length = 20):
    if (debugMode):
        percent = float(value) / endvalue
        arrow = '-' * int(round(percent * bar_length)-1) + '>'
        spaces = ' ' * (bar_length - len(arrow))
        sys.stdout.write("\r" 
                         + str(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")) 
                         + " | " 
                         + message 
                         + ": [{0}] {1}%".format(arrow + spaces, int(round(percent * 100)))
                        )
        if value == endvalue:
            sys.stdout.write("\n")
        sys.stdout.flush()

In [6]:
def get_hdfs_client():
    return InsecureClient("emma0.emma.nlesc.nl:50070", user="pheno",
         root="/")

### Read functions
These functions allow for the reading of data.

In [7]:
def getDataSet(directoryPath, bandNum):
    dprint("Running getDataSet(directoryPath)")
    
    files = sc.binaryFiles(directoryPath + "/*.tif")
    fileList = files.keys().collect()
    dprint("Number of files: " + str(len(fileList)))
    dataSet = []
    plotShapes = []
    flattenedShapes = []
    for i, f in enumerate(fileList):
        progressBar("Reading files", i + 1, len(fileList))
        data = files.lookup(f)
        dataByteArray = bytearray(data[0])
        memfile = MemoryFile(dataByteArray)
        dataset = memfile.open()
        relevantBand = np.array(dataset.read()[bandNum])
        memfile.close()
        plotShapes.append(relevantBand.shape)
        flattenedDataSet = relevantBand.flatten()
        flattenedShapes.append(flattenedDataSet.shape)
        dataSet.append(flattenedDataSet)
    dataSet = np.array(dataSet).T
    dprint("dataSet.shape: " + str(dataSet.shape))
    
    dprint("Ending getDataSet(directoryPath)")
    return dataSet

In [8]:
def getMask(filePath):
    dprint("Running getMask(filePath)")
    
    mask_data = sc.binaryFiles(filePath).take(1)
    mask_byteArray = bytearray(mask_data[0][1])
    mask_memfile = MemoryFile(mask_byteArray)
    mask_dataset = mask_memfile.open()
    maskTransform = mask_dataset.transform
    mask_data = np.array(mask_dataset.read()[0])
    mask_memfile.close()
    dprint("mask_data.shape: " + str(mask_data.shape))
    
    dprint("Ending getMask(filePath)")
    return mask_data, maskTransform

### Utility functions
These functions analyse and manipulate data.

In [9]:
def filterDataSet(dataSet, maskData):
    dprint("Running filterDataSet(dataSet, maskIndex)")
    
    maskIndex = np.nonzero(maskData.flatten())[0]
    dataSetFiltered = dataSet[maskIndex]
    dprint("dataSetFiltered.shape: " + str(dataSetFiltered.shape))
    
    dprint("Ending filterDataSet(dataSet, maskIndex)")
    return dataSetFiltered

In [10]:
def validateNorms(dataSet1, dataSet2, U, s, V):
    dprint("Running validateNorms(dataSet1, dataSet2, U, s, V)")
    
    length = len(s)
    norms = []
    for i in range(length):
        progressBar("Validating norms", i + 1, length)
        u = dataSet1 @ (dataSet2.T @ V.T[i]) / s[i]
        v = dataSet2 @ (dataSet1.T @ U.T[i]) / s[i]
        norms.append(scipy.linalg.norm(U.T[i] - u))
        norms.append(scipy.linalg.norm(V.T[i] - v))
    dprint("Largest norm difference: " + str(max(norms)))
    
    dprint("Ending validateNorms(dataSet1, dataSet2, U, s, V)")

### Write functions
These functions write data and plots.

In [11]:
def writeCSVs(resultDirectory, U, s, V):
    dprint("Running writeCSV(resultDirectory, U, s, V)")
    
    for i, vectorData in enumerate([U, s, V]):
        progressBar("Writing CSV", i + 1, 3)
        fileName = ["U", "s", "V"][i] + ".csv"
        inFile = "/tmp/" + fileName
        outFile = resultDirectory + fileName
        decompositionFile = open(inFile, "w")
        vectorData.T.tofile(decompositionFile, sep = ",")
        decompositionFile.close()
        #Upload to HDFS
        subprocess.run(['hadoop', 'dfs', '-copyFromLocal', '-f', inFile, outFile])
        #Remove from /tmp/
        subprocess.run(['rm', '-fr', inFile])
    
    dprint("Ending writeCSV(resultDirectory, U, s, V)")

In [12]:
def plotSingularValues(resultDirectory, s):
    dprint("Running plotSingularValues(resultDirectory, s)")
    
    fileName = "s.pdf"
    inFile = "/tmp/" + fileName
    outFile = resultDirectory + fileName
    x = range(len(s))
    total = s.T @ s
    cumulativeValue = 0
    valueList = []
    cumulativeList = []
    for i in x:
        value = np.square(s[i]) / total
        valueList.append(value)
        cumulativeValue = cumulativeValue + value
        cumulativeList.append(cumulativeValue)
    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()
    ax1.plot(x, valueList, "g^")
    ax2.plot(x, cumulativeList, "ro")
    ax1.set_xlabel("Singular values")
    ax1.set_ylabel("Variance explained", color = "g")
    ax2.set_ylabel("Cumulative variance explained", color = "r")
    plt.savefig(inFile)
    plt.clf()
    #Upload to HDFS
    subprocess.run(['hadoop', 'dfs', '-copyFromLocal', '-f', inFile, outFile])  
    #Remove from /tmp/
    subprocess.run(['rm', '-fr', inFile])
    
    dprint("Ending plotSingularValues(resultDirectory, s)")

In [13]:
def writeModes(resultDirectory, U, s, V):
    dprint("Running writeModes(resultDirectory, U, s, V)")
    
    for i in range(len(s)):
        progressBar("Writing modes", i + 1, len(s))
        fileName = "Mode" + str(i + 1).zfill(2) + ".txt"
        inFile = "/tmp/" + fileName
        outFile = resultDirectory + fileName
        decompositionFile = open(inFile, "w")
        U.T[i].tofile(decompositionFile, sep = ",")
        decompositionFile.close()
        decompositionFile = open(inFile, "a")
        decompositionFile.write("\n")
        s[i].tofile(decompositionFile, sep = ",")
        decompositionFile.write("\n")
        V.T[i].tofile(decompositionFile, sep = ",")
        decompositionFile.close()
        #Upload to HDFS
        subprocess.run(['hadoop', 'dfs', '-copyFromLocal', '-f', inFile, outFile])  
        #Remove from /tmp/
        subprocess.run(['rm', '-fr', inFile])
    
    dprint("Ending writeModes(resultDirectory, U, s, V)")

In [19]:
def plotModes(resultDirectory, U, s, V, maskData, maskTransform):
    dprint("Running plotModes(resultDirectory, U, s, V, maskData, maskTransform)")
    
    plotTemplate = np.full(maskData.shape[0] * maskData.shape[1], np.nan, dtype=np.float64)
    maskIndex = np.nonzero(maskData.flatten())[0]
        
    for i in range(min(maxModes, len(s))):
        progressBar("Plotting modes", i + 1, min(maxModes, len(s)))
        for vectorData, vectorName in zip([U, V], ["U", "V"]):
            data = np.copy(plotTemplate)
            np.put(data, maskIndex, vectorData.T[i])
            data = np.reshape(data, maskData.shape, )
                        
            fileName = "Mode" + vectorName + str(i + 1).zfill(2) + ".tif"
            inFile = "/tmp/" + fileName
            outFile = resultDirectory + fileName
            rasterioPlot = rasterio.open(inFile, "w", driver = "GTiff", width = data.shape[1], height = data.shape[0], count = 1, dtype = data.dtype, crs = "EPSG:4326", transform = maskTransform, compress="deflate")
            rasterioPlot.write(data, 1)
            rasterioPlot.close()
            #Upload to HDFS
            subprocess.run(['hadoop', 'dfs', '-copyFromLocal', '-f', inFile, outFile])  
            #Remove from /tmp/
            subprocess.run(['rm', '-fr', inFile])
    
    dprint("Ending plotModes(resultDirectory, U, s, V, maskData, maskTransform)")

### Analysis function
This function combines all the necessary steps for the analysis.

In [20]:
def runAnalysis(dataDirectory1, dataDirectory2, bandNum1, bandNum2, maskFile, resultDirectory):
    dprint("Running runAnalysis(dataDirectory1, dataDirectory2, maskFile, resultDirectory)")

    dataSet1 = getDataSet(dataDirectory1, bandNum1)
    dataSet2 = getDataSet(dataDirectory2, bandNum2)
    
    if (dataSet2.shape[1] == 26 and dataSet1.shape[1] != 26): # Hack to align time-dimension of SOS with Bloom and Leaf
        dataSet1 = dataSet1[:, 8:34]
    
    maskData, maskTransform = getMask(maskFile)
    
    dataSetFiltered1 = filterDataSet(dataSet1, maskData)
    dataSetFiltered2 = filterDataSet(dataSet2, maskData)
    
    U, s, Vt = qrproductsvd(dataSetFiltered1, dataSetFiltered2)
    V = Vt.T
    dprint("U.shape: " + str(U.shape))
    dprint("s.shape: " + str(s.shape))
    dprint("V.shape: " + str(V.shape))
    dprint("Singular values of product: ")
    dprint(str(s))
    
    validateNorms(dataSetFiltered1, dataSetFiltered2, U, s, V)
    
    plotSingularValues(resultDirectory, s)
    writeModes(resultDirectory, U, s, V)
    plotModes(resultDirectory, U, s, V, maskData, maskTransform)
    writeCSVs(resultDirectory, U, s, V)
    
    dprint("Ending runAnalysis(dataDirectory1, dataDirectory2, maskFile, resultDirectory)")

## Analyses
In this section, the various analyses are initiated. Each analysis uses a different pair of datasets.

### Analysis 1
This analysis focusses on Bloom and Leaf data from the USA from 1980 to 2016 at a 4K spatial resolution.

In [21]:
dprint("-------------------------------")
dprint("Running analysis 1")
dprint("-------------------------------")

dataDirectory1 = "hdfs:///user/hadoop/spring-index/BloomGridmet/"
bandNum1 = 3
dataDirectory2 = "hdfs:///user/hadoop/spring-index/LeafGridmet/"
bandNum2 = 3
maskFile = "hdfs:///user/hadoop/usa_mask_gridmet.tif"
resultDirectory = "hdfs:///user/emma/svd/BloomGridmetLeafGridmet/"

#Create Result dir
subprocess.run(['hadoop', 'dfs', '-mkdir', resultDirectory])

runAnalysis(dataDirectory1, dataDirectory2, bandNum1, bandNum2, maskFile, resultDirectory)

dprint("-------------------------------")
dprint("Ending analysis 1")
dprint("-------------------------------")

2017-12-12 14:52:38 | -------------------------------
2017-12-12 14:52:38 | Running analysis 1
2017-12-12 14:52:38 | -------------------------------
2017-12-12 14:52:40 | Running runAnalysis(dataDirectory1, dataDirectory2, maskFile, resultDirectory)
2017-12-12 14:52:40 | Running getDataSet(directoryPath)
2017-12-12 14:52:45 | Number of files: 37
2017-12-12 14:53:34 | Reading files: [------------------->] 100%
2017-12-12 14:53:36 | dataSet.shape: (1414560, 37)
2017-12-12 14:53:36 | Ending getDataSet(directoryPath)
2017-12-12 14:53:36 | Running getDataSet(directoryPath)
2017-12-12 14:53:38 | Number of files: 37
2017-12-12 14:54:41 | Reading files: [------------------->] 100%
2017-12-12 14:54:43 | dataSet.shape: (1414560, 37)
2017-12-12 14:54:43 | Ending getDataSet(directoryPath)
2017-12-12 14:54:43 | Running getMask(filePath)
2017-12-12 14:54:44 | mask_data.shape: (840, 1684)
2017-12-12 14:54:44 | Ending getMask(filePath)
2017-12-12 14:54:44 | Running filterDataSet(dataSet, maskIndex)
20

### Analysis 2
This analysis focusses on Bloom and SOS data from the USA from 1980 to 2016 at a 4K spatial resolution.

In [ ]:
dprint("-------------------------------")
dprint("Running analysis 2")
dprint("-------------------------------")

dataDirectory1 = "hdfs:///user/hadoop/spring-index/BloomGridmet/"
bandNum1 = 3
dataDirectory2 = "hdfs:///user/hadoop/avhrr/SOST4Km/"
bandNum2 = 0
maskFile = "hdfs:///user/hadoop/usa_mask_gridmet.tif"
resultDirectory = "hdfs:///user/emma/svd/BloomGridmetSOST4Km/"

#Create Result dir
subprocess.run(['hadoop', 'dfs', '-mkdir', resultDirectory])

runAnalysis(dataDirectory1, dataDirectory2, bandNum1, bandNum2, maskFile, resultDirectory)

dprint("-------------------------------")
dprint("Ending analysis 2")
dprint("-------------------------------")

### Analysis 3
This analysis focusses on Leaf and SOS data from the USA from 1980 to 2016 at a 4K spatial resolution.

In [ ]:
dprint("-------------------------------")
dprint("Running analysis 3")
dprint("-------------------------------")

dataDirectory1 = "hdfs:///user/hadoop/spring-index/LeafGridmet/"
bandNum1 = 3
dataDirectory2 = "hdfs:///user/hadoop/avhrr/SOST4Km/"
bandNum2 = 0
maskFile = "hdfs:///user/hadoop/usa_mask_gridmet.tif"
resultDirectory = "hdfs:///user/emma/svd/LeafGridmetSOST4Km/"

#Create Result dir
subprocess.run(['hadoop', 'dfs', '-mkdir', resultDirectory])

runAnalysis(dataDirectory1, dataDirectory2, bandNum1, bandNum2, maskFile, resultDirectory)

dprint("-------------------------------")
dprint("Ending analysis 3")
dprint("-------------------------------")

### Analysis 4
This analysis focusses on BloomFinalLowPR and SOSTLowPR data from the USA from 1989 to 2014 1Km resolution.

In [19]:
dprint("-------------------------------")
dprint("Running analysis 4")
dprint("-------------------------------")

dataDirectory1 = "hdfs:///user/hadoop/spring-index/BloomFinalLowPR/"
bandNum1 = 0
dataDirectory2 = "hdfs:///user/hadoop/avhrr/SOSTLowPR/"
bandNum2 = 0
maskFile = "hdfs:///user/hadoop/spring-index/BloomFinalLowPR/1989.tif"
resultDirectory = "hdfs:///user/emma/svd/BloomFinalLowPRSOSTLowPR/"

#Create Result dir
subprocess.run(['hadoop', 'dfs', '-mkdir', resultDirectory])

runAnalysis(dataDirectory1, dataDirectory2, bandNum1, bandNum2, maskFile, resultDirectory)

dprint("-------------------------------")
dprint("Ending analysis 4")
dprint("-------------------------------")

2017-12-12 10:48:05 | -------------------------------
2017-12-12 10:48:05 | Running analysis 4
2017-12-12 10:48:05 | -------------------------------
2017-12-12 10:48:07 | Running runAnalysis(dataDirectory1, dataDirectory2, maskFile, resultDirectory)
2017-12-12 10:48:07 | Running getDataSet(directoryPath)
2017-12-12 10:48:11 | Number of files: 26
2017-12-12 10:48:37 | Reading files: [------------------->] 100%
2017-12-12 10:48:38 | dataSet.shape: (30870, 26)
2017-12-12 10:48:38 | Ending getDataSet(directoryPath)
2017-12-12 10:48:38 | Running getDataSet(directoryPath)
2017-12-12 10:48:39 | Number of files: 26
2017-12-12 10:48:58 | Reading files: [------------------->] 100%
2017-12-12 10:48:59 | dataSet.shape: (30870, 26)
2017-12-12 10:48:59 | Ending getDataSet(directoryPath)
2017-12-12 10:48:59 | Running getMask(filePath)
2017-12-12 10:48:59 | mask_data.shape: (210, 147)
2017-12-12 10:48:59 | Ending getMask(filePath)
2017-12-12 10:48:59 | Running filterDataSet(dataSet, maskIndex)
2017-12

### Analysis 5
This analysis focusses on LeafFinalLowPR and SOSTLowPR data from the USA from 1989 to 2014 1Km resolution.

In [20]:
dprint("-------------------------------")
dprint("Running analysis 5")
dprint("-------------------------------")

dataDirectory1 = "hdfs:///user/hadoop/spring-index/LeafFinalLowPR/"
bandNum1 = 0
dataDirectory2 = "hdfs:///user/hadoop/avhrr/SOSTLowPR/"
bandNum2 = 0
maskFile = "hdfs:///user/hadoop/spring-index/LeafFinalLowPR/1989.tif"
resultDirectory = "hdfs:///user/emma/svd/LeafFinalLowPRSOSTLowPR/"

#Create Result dir
subprocess.run(['hadoop', 'dfs', '-mkdir', resultDirectory])

runAnalysis(dataDirectory1, dataDirectory2, bandNum1, bandNum2, maskFile, resultDirectory)

dprint("-------------------------------")
dprint("Ending analysis 5")
dprint("-------------------------------")

2017-12-12 10:50:34 | -------------------------------
2017-12-12 10:50:34 | Running analysis 5
2017-12-12 10:50:34 | -------------------------------
2017-12-12 10:50:36 | Running runAnalysis(dataDirectory1, dataDirectory2, maskFile, resultDirectory)
2017-12-12 10:50:36 | Running getDataSet(directoryPath)
2017-12-12 10:50:44 | Number of files: 26
2017-12-12 10:51:07 | Reading files: [------------------->] 100%
2017-12-12 10:51:08 | dataSet.shape: (30870, 26)
2017-12-12 10:51:08 | Ending getDataSet(directoryPath)
2017-12-12 10:51:08 | Running getDataSet(directoryPath)
2017-12-12 10:51:09 | Number of files: 26
2017-12-12 10:51:30 | Reading files: [------------------->] 100%
2017-12-12 10:51:31 | dataSet.shape: (30870, 26)
2017-12-12 10:51:31 | Ending getDataSet(directoryPath)
2017-12-12 10:51:31 | Running getMask(filePath)
2017-12-12 10:51:31 | mask_data.shape: (210, 147)
2017-12-12 10:51:31 | Ending getMask(filePath)
2017-12-12 10:51:31 | Running filterDataSet(dataSet, maskIndex)
2017-12

### Analysis 6
This analysis focusses on BloomFinalLowPR and LeafFinalLowPR data from the USA from 1989 to 2014 1Km resolution.

In [21]:
dprint("-------------------------------")
dprint("Running analysis 6")
dprint("-------------------------------")

dataDirectory1 = "hdfs:///user/hadoop/spring-index/BloomFinalLowPR/"
bandNum1 = 0
dataDirectory2 = "hdfs:///user/hadoop/spring-index/LeafFinalLowPR/"
bandNum2 = 0
maskFile = "hdfs:///user/hadoop/spring-index/BloomFinalLowPR/1989.tif"
resultDirectory = "hdfs:///user/emma/svd/BloomFinalLowPRLeafFinalLowPR/"

#Create Result dir
subprocess.run(['hadoop', 'dfs', '-mkdir', resultDirectory])

runAnalysis(dataDirectory1, dataDirectory2, bandNum1, bandNum2, maskFile, resultDirectory)

dprint("-------------------------------")
dprint("Ending analysis 6")
dprint("-------------------------------")

2017-12-12 10:53:05 | -------------------------------
2017-12-12 10:53:05 | Running analysis 6
2017-12-12 10:53:05 | -------------------------------
2017-12-12 10:53:07 | Running runAnalysis(dataDirectory1, dataDirectory2, maskFile, resultDirectory)
2017-12-12 10:53:07 | Running getDataSet(directoryPath)
2017-12-12 10:53:15 | Number of files: 26
2017-12-12 10:53:39 | Reading files: [------------------->] 100%
2017-12-12 10:53:40 | dataSet.shape: (30870, 26)
2017-12-12 10:53:40 | Ending getDataSet(directoryPath)
2017-12-12 10:53:40 | Running getDataSet(directoryPath)
2017-12-12 10:53:41 | Number of files: 26
2017-12-12 10:54:03 | Reading files: [------------------->] 100%
2017-12-12 10:54:04 | dataSet.shape: (30870, 26)
2017-12-12 10:54:04 | Ending getDataSet(directoryPath)
2017-12-12 10:54:04 | Running getMask(filePath)
2017-12-12 10:54:04 | mask_data.shape: (210, 147)
2017-12-12 10:54:04 | Ending getMask(filePath)
2017-12-12 10:54:04 | Running filterDataSet(dataSet, maskIndex)
2017-12

End of Notebook